In [ ]:
!pip install requests tensorflow pandas numpy matplotlib scikit-learn


In [ ]:
import requests
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
# 🔹 Replace with your actual Blynk Auth Token
BLYNK_AUTH_TOKEN = "qf-xfmiwprNBw0JCVKMw4YiJs83vqQ6c"
BLYNK_API_URL = f"https://blynk.cloud/external/api/get?token={BLYNK_AUTH_TOKEN}&V0"

# Function to fetch voltage values from Blynk API
def get_voltage():
    """Fetch real-time voltage from Blynk API."""
    try:
        response = requests.get(BLYNK_API_URL)
        if response.status_code == 200:
            return float(response.text)
        else:
            print("Error fetching data from Blynk API")
            return None
    except Exception as e:
        print("Request failed:", e)
        return None


In [ ]:
# Load data
df = pd.read_csv("/content/voltage_data .csv")

# Normalize the data (LSTM requires normalization)
scaler = MinMaxScaler(feature_range=(0, 1))
df["Voltage"] = scaler.fit_transform(df[["Voltage"]])

# Convert the data into sequences for LSTM
def create_sequences(data, seq_length):
    sequences = []
    labels = []

    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        label = data[i+seq_length]
        sequences.append(seq)
        labels.append(label)

    return np.array(sequences), np.array(labels)

seq_length = 10  # Use past 10 readings to predict the next one
X, y = create_sequences(df["Voltage"].values, seq_length)

# Reshape X for LSTM input (samples, time steps, features)
X = X.reshape(X.shape[0], X.shape[1], 1)


FileNotFoundError: [Errno 2] No such file or directory: '/content/voltage_data .csv'

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Build LSTM Model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(seq_length, 1)),
    LSTM(50, return_sequences=False),
    Dense(25, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X, y, epochs=20, batch_size=8)


Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.5251
Epoch 2/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0274
Epoch 3/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0131
Epoch 4/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0054
Epoch 5/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0062
Epoch 6/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0047
Epoch 7/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0040
Epoch 8/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0049
Epoch 9/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0065
Epoch 10/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0048
Epoch 11/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0040
Epoch 12/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0042
Epoch 13/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0045
Epoch 14/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0048
Epoch 15/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0035
Epoch 16/20
12/

In [ ]:
# Threshold for high voltage
VOLTAGE_THRESHOLD = 235.0  # 240V is the limit

# Function to classify voltage levels
def classify_voltage(voltage):
    if voltage > VOLTAGE_THRESHOLD:
        return "High (Fault)"
    else:
        return "Low (Normal)"


In [ ]:
# Test the model in real-time
while True:
    voltage = get_voltage()

    if voltage is not None:
        # Classify the voltage as High (Fault) or Low (Normal)
        status = classify_voltage(voltage)
        print(f"Voltage: {voltage}V, Status: {status}")

        # If voltage is high, send alert
        if status == "High (Fault)":
            send_alert(f"⚠️ Warning! High Voltage: {voltage}V")

    time.sleep(5)  # Fetch new data every 5 seconds


Voltage: 242.28V, Status: High (Fault)
⚠️ Failed to send alert.
Voltage: 241.73V, Status: High (Fault)
⚠️ Failed to send alert.
Voltage: 242.28V, Status: High (Fault)
⚠️ Failed to send alert.
Voltage: 242.0V, Status: High (Fault)
⚠️ Failed to send alert.
Voltage: 243.1V, Status: High (Fault)
⚠️ Failed to send alert.
Voltage: 242.28V, Status: High (Fault)
⚠️ Failed to send alert.
Voltage: 242.0V, Status: High (Fault)
⚠️ Failed to send alert.
Voltage: 212.47V, Status: Low (Normal)
Voltage: 242.28V, Status: High (Fault)
⚠️ Failed to send alert.
Voltage: 241.73V, Status: High (Fault)
⚠️ Failed to send alert.
Voltage: 241.18V, Status: High (Fault)
⚠️ Failed to send alert.
Voltage: 200.99V, Status: Low (Normal)
Voltage: 218.49V, Status: Low (Normal)
Voltage: 217.67V, Status: Low (Normal)
Voltage: 224.5V, Status: Low (Normal)
Voltage: 235.72V, Status: High (Fault)
⚠️ Failed to send alert.
Voltage: 229.97V, Status: Low (Normal)
Voltage: 0.0V, Status: Low (Normal)
Voltage: 0.0V, Status: Low (No

KeyboardInterrupt: 